<img src="Tutorial-DirectStitch-figs/Chameleon-FullColor-lg_4CCULFA.jpg" >

# Tutorial: Using Chameleon DirectStitch Connections 

- **Estimated time**: ?? minutes
- **Requirements**:

     - This tutorial assumes you have a Chameleon account and basic experience logging into and using Chameleon (see: https://chameleoncloud.readthedocs.io/en/latest/getting-started/index.html)
     - Understanding of Chameleon networks and isolated VLANs. (see: https://chameleoncloud.readthedocs.io/en/latest/technical/networks.html)
     - Basic understanding of the CLI (see https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html)
     - Have a GENI account and SSL Certificate (see http://www.exogeni.net/2015/09/exogeni-getting-started-tutorial/)

The Chameleon DirectStitch capability provides direct OSI Layer 2 networking connections between Chameleon resources and external facilities. Chameleon DirectStitch is similar services offered by public cloud providers such as AWS Direct Connect, Google Cloud Dedicated Interconnect, and Microsoft Azure ExpressRoute. 



During this tutorial we will:

- Step 1: Create Chameleon Isolated Network(s) with DirectStitch Ports](#Step-1%3A-Do-something) 
- Step 2: Use ExoGENI to Connect the Ports](#Step-2%3A-Do-something-else)
- Step 3: Run an Experiment using the Wide-area Layer 2 Connection](#Step-3%3A-Do-something-else)


## Background: Direct Network Connections to Public Clouds

Many public cloud providers have services for connecting campus and business IT facilites directly to cloud resources over OSI layer 2 connections. These connections can be more reliable, lower latency, higher bandwidth, and more secure. In addition, public cloud providers often provide discounts on data transfers made using dedicated infrastructure as opposed to comodity Internet. Although they are similar, each public cloud provider has its own interface and set of capabilities. 

Direct network connection services provided by common public clouds: 

- AWS Direct Connect: https://aws.amazon.com/directconnect/
- Google Cloud Dedicated Interconnect: https://cloud.google.com/interconnect/docs/
- Microsoft Azure ExpressRoute: https://docs.microsoft.com/en-us/azure/expressroute/expressroute-introduction
    
Internet2 has simplified access to many of these services through Cloud Connect (https://www.internet2.edu/products-services/advanced-networking/networking-for-cloud/) which is available to Internet2's member institutions.


<img src="Tutorial-DirectStitch-figs/direct_connect_overview.png" width="500">


An example can be seen in Amazons Direct Connect service illistrated in the figure above. Users can connect their campus router or firewall (i.e. "customer network") to various AWS peering points (i.e. "AWS Direct Connect locations").  At the same time users can connect their Virtual Private Cloud (VPC) and its EC2 nodes to the peering point and have traffic forwared directly between their campus and resources in the Amazon cloud.
 
### Why use direct cloud network connections?

There are several common production use cases for direct cloud network connetions:

   - Extending Campus Infrastructure. With direct connections a campus infrastructure can be extended into the cloud while making it appear to be local (i.e. cloud resources using local capmus IP space). Your campus may already be doing this without you knowing.
   - Cloud bursting. Dynamically extending campus infrastructure to the cloud during periods of peak demand. 
   - Network Performance: Dedicated direct connections can have high bandwidth and/or lower latency compared to comodity Internet.
   - Security: Directly connected cloud resources are not exposed to traffic from the common IP address name space shared across the Internet. This can be used to create an increased layer of security for increasingly common friction free network paths used to transfer science data (e.g. ScienceDMZs).
   
Educational and research institutions can use direct connections to create sandboxes for real wide-area network eperiments:

   - Network peering. Students can deploy BGP on isolated networks spread across the world. Researchers can experiements with alternative protocols and archetectures.
   - Performance. Students and researchers can experiment with realistic wide-area network performance with increased control of the core  network.
   - Non-TCP/IP protocols. Students and researchers can experiment with non-TCP/IP protocols in the real wide-area network. Note that this is not possile with all public cloud providers but it is possible with Chameleon. 

## Chameleon DirectStitch
  
Chameleon's DirectStitch capabilites enables direct OSI layer 2 connections and the associated wide-area networking experiments described in the previous section to a cloud that is more deeply programmable than then common public clouds. 
  
<img src="Tutorial-DirectStitch-figs/ChameleonDirectStitch.png" width="500">
  
A Chameleon Isolated Tenant Network has similar properties to an Amazon VPC. Fudementally, it is a private network for connecting cloud nodes. Typically, each node is assigned a fixed IP on the private network though which they can communicate with other nodes on the network and uses a NAT router managed by the Chameleon's OpenStack system to commuicate with the public Internet. Optionally, nodes can be provide services on the pubic Internet by being assigned a floating IP that the NAT router forward to the specfic node.  

Chameleon DirectStitch works by adding a port to the isolated network's switch that connects connects to Internet2's Advanced Layer 2 Service (AL2S) on a specific VLAN tag. The user can then use AL2S to connect a layer 2 circut between the Chameleon DirectStitch port and any other endpoint on AL2S.  

The major limiation to using DirectStitch, as well as other public cloud direct network connection services, is that most users do not have direct access to a wide-area network circuit provider such as AL2S.  The solution for academic educators and reseachers is to use GENI as an intermediary to provision these circuits.   GENI users can use ExoGENI to provision circuits between GENI resources and other AL2S endpoints including isolated tenant networks on Chameleon and, in the near future, other public cloud direct connect locations enabled by Internet2 Cloud Connnect.

For information about enabling connections to your campus facility contact the Chameleon or ExoGENI team. We are always interested in connecting more facilities and enabling more experiments.



## Tutorial

In this tutorial, you will use Chameleon DirectStitch ports to connect isolated tenant networks at each of the two Chameleon sites.  A similar process can be used to connect an isolated tenant network to your campus facility or other resource on ExoGEN.

<img src="Tutorial-DirectStitch-figs/ChameleonDirectStitch-TACC2UC.png" width="700">


### Variables you'll see/use in this Notebook

  - `OS_PROJECT_NAME`: The name of the project you want to use.
  - `UC_RESERVATION`: Reservation ID at UC.
  - `TACC_RESERVATION`: Reservation ID at TACC.
  - `SSH_KEY_NAME`: SSH key name as registered in Chameleon
  - `SSH_PRIVATE_KEY`: Path to the corresponding private key in your Jupyter container.
  - `NETWORK_SUBNET`: Private IP subnet that will span both Chameleon sites.
  - `UC_ALLOCATION_START`: The start of the IP range assigned at the UC site.
  - `UC_ALLOCATION_END`: The end of the IP range assigned at the UC site.
  - `TACC_ALLOCATION_START`: The start of the IP range assigned at the TACC site.
  - `TACC_ALLOCATION_END`: The start of the IP range assigned at the TACC site.
  - `RESOURCE_PREFIX`: A begin string used to idenify resources provision. This includes your user ID so that your resources can be distinguised from those belonging to other tutorial particpants.
  - `GENI_PEM_FILE`: The path to your GENI certificate file in you Jupyter container.
  - `UC_DIRECTSTITCH_VLAN`: The VLAN using to attach to the DirectStitch port at UC.
  - `TACC_DIRECTSTITCH_VLAN`: The VLAN using to attach to the DirectStitch port at TACC.

### Set Variables

In [1]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='CH-816532'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-jupyter
export SSH_PRIVATE_KEY=${HOME}/work/pruth-chameleon-jupyter

export NETWORK_SUBNET="192.168.100.0/24"
export UC_ALLOCATION_START="192.168.100.10"
export UC_ALLOCATION_END="192.168.100.19"
export TACC_ALLOCATION_START="192.168.100.20"
export TACC_ALLOCATION_END="192.168.100.29"

export RESOURCE_PREFIX="$USER-tutorial-$(date +%b%d)"

export GENI_PEM_FILE="~/work/geni-pruth.pem"


## Step 1: Create Chameleon Isolated Network(s) with DirectStitch Ports

Creating a Chameleon DirectStitch port is as easy as creating an isolated VLAN but with an extra parameter requesting the port. Unlike local isolated VLANs, there are a limited number of Chameleon DirectStitch ports available. In order to share access to ports, Chameleon requires a DirectStitch reservation.

In this step we will create a two Chameleon networks each with a DirectConnect port. One network will be at the UC (Chicago) while the other will be at TACC (Austin).

### Step 1a: Create a Network a UC

#### Set network resource names

In [2]:
UC_RESERVATION_NAME=${RESOURCE_PREFIX}"-UC"

# Set the names for the network, subnet, router, and switch. 
UC_NETWORK_NAME=${RESOURCE_PREFIX}"-Network-UC"
UC_SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet-UC"
UC_ROUTER_NAME=${RESOURCE_PREFIX}"-Router-UC"



#### Create the Network

DirectStitch networks must be researved using the reservation system.  In this reservation, we  include a compute node that we will use later in the tutorial.  

Note: that you can make separate reservations for nodes and networks.

In [3]:
OS_REGION_NAME='CHI@UC'

START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${EXOGENI_STITCH_NAME}
blazar lease-create \
   --physical-reservation min=1,max=1,resource_properties='["=", "$node_type", "compute_haswell"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${UC_NETWORK_NAME}",resource_properties='["==","$physical_network","exogeni"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${UC_RESERVATION_NAME}

UC_RESERVATION=`blazar lease-show --format value -c id ${UC_RESERVATION_NAME}`
echo UC_RESERVATION $UC_RESERVATION

UC_NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${UC_RESERVATION_NAME} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo UC_NODE_RESERVATION $UC_NODE_RESERVATION


Creating network
Matches: ('min=1,max=1', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Matches: ('resource_type=virtual:floatingip,network_id=44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '44b38c44-2a42-4b6d-b129-6c8f1b2a1375')
Matches: (None, 'resource_type', 'virtual:floatingip')
Matches: ('resource_type=network,network_name=pruth-tutorial-May30-Network-UC', 'resource_properties', '["==","$physical_network","exogeni"]')
Matches: ('resource_type=network', 'network_name', 'pruth-tutorial-May30-Network-UC')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+------------------------------------------------------------------------

#### Get the DirectStitch VLAN Tag

Note: this will not be successful until the network is created. You may need to rerun this command until the VLAN tag is found.

In [6]:
OS_REGION_NAME='CHI@UC'

UC_DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${UC_NETWORK_NAME}`
echo UC DirectStitch VLAN: $UC_DIRECTSTITCH_VLAN 

UC_FLOATING_IP=`lease_list_floating_ips $UC_RESERVATION`
echo UC_FLOATING_IP $UC_FLOATING_IP 


UC DirectStitch VLAN: 3293
UC_FLOATING_IP 192.5.87.223


#### Step 1b: Add a subnet and router to the network

In [7]:
OS_REGION_NAME='CHI@UC'
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --allocation-pool start=${UC_ALLOCATION_START},end=${UC_ALLOCATION_END} \
                        --dhcp \
                        --network ${UC_NETWORK_NAME} \
                        ${UC_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${UC_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${UC_ROUTER_NAME} ${UC_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${UC_ROUTER_NAME}

echo Network ${UC_NETWORK_NAME} is ready for nodes!

Creating Subnet
BadRequestException: 400: Client Error for url: https://chi.uc.chameleoncloud.org:9696/v2.0/subnets, Invalid input for operation: Requested subnet with cidr: 192.168.100.0/24 for network: af8f013e-d034-455f-a269-c3aa5db5da40 overlaps with another subnet.
Creating Router
+-------------------------+----------------------------------------------------+
| Field                   | Value                                              |
+-------------------------+----------------------------------------------------+
| admin_state_up          | UP                                                 |
| availability_zone_hints |                                                    |
| availability_zones      |                                                    |
| created_at              | 2019-05-30T01:44:04Z                               |
| description             |                                                    |
| external_gateway_info   | None                                 

#### Step 1c: Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [8]:
UC_NODE_NAME=${RESOURCE_PREFIX}-node

OS_REGION_NAME='CHI@UC'
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${UC_NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${UC_NETWORK_NAME} \
                        ${UC_NODE_NAME}

Creating servers... This will take several minutes!

+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         | nova                                   |
| OS-EXT-SRV-ATTR:host                | admin01-ironic                         |
| OS-EXT-SRV-ATTR:hypervisor_hostname | 15a37657-65bb-4a39-8f2d-c308e7226651   |
| OS-EXT-SRV-ATTR:instance_name       | instance-000087f2                      |
| OS-EXT-STS:power_state              | Running                                |
| OS-EXT-STS:task_state               | None                                   |
| OS-EXT-STS:vm_state                 | active                                 |
| OS-SRV-USG:launched_at              | 2019-05-30T01:52

In [9]:
OS_REGION_NAME='CHI@UC'
#Attach the floating IP address to the server
echo $UC_FLOATING_IP 
openstack server add floating ip $UC_NODE_NAME $UC_FLOATING_IP 

192.5.87.223


### Create Network a TACC

Repeat the steps but target the TACC Chameleon site

In [10]:
OS_REGION_NAME='CHI@TACC'

TACC_RESERVATION_NAME=${RESOURCE_PREFIX}"-TACC"

# Set the names for the network, subnet, router, and switch. 
# See above about using identifiable names.  
TACC_NETWORK_NAME=${RESOURCE_PREFIX}"-Network-TACC"
TACC_SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet-TACC"
TACC_ROUTER_NAME=${RESOURCE_PREFIX}"-Router-TACC"

#### Create the Network

In [11]:
OS_REGION_NAME='CHI@TACC'

START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+1 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${EXOGENI_STITCH_NAME}
blazar lease-create \
   --physical-reservation min=1,max=1,resource_properties='["=", "$node_type", "compute_haswell"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${TACC_NETWORK_NAME}",resource_properties='["==","$physical_network","exogeni"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${TACC_RESERVATION_NAME}

TACC_RESERVATION=`blazar lease-show --format value -c id ${TACC_RESERVATION_NAME}`
echo TACC_RESERVATION $TACC_RESERVATION

TACC_NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${TACC_RESERVATION} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo TACC_NODE_RESERVATION $TACC_NODE_RESERVATION
   
   
   

Creating network
Matches: ('min=1,max=1', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '1')
Matches: (None, 'min', '1')
Matches: ('resource_type=virtual:floatingip,network_id=6189521e-06a0-4c43-b163-16cc11ce675b', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '6189521e-06a0-4c43-b163-16cc11ce675b')
Matches: (None, 'resource_type', 'virtual:floatingip')
Matches: ('resource_type=network,network_name=pruth-tutorial-May30-Network-TACC', 'resource_properties', '["==","$physical_network","exogeni"]')
Matches: ('resource_type=network', 'network_name', 'pruth-tutorial-May30-Network-TACC')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+--------------------------------------------------------------------

#### Get the DirectStitch VLAN Tag

Repeat until sucessful

In [12]:
OS_REGION_NAME='CHI@TACC'
TACC_DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${TACC_NETWORK_NAME}`
echo TACC DirectStitch VLAN: $TACC_DIRECTSTITCH_VLAN 

TACC_FLOATING_IP=`lease_list_floating_ips $TACC_RESERVATION`
echo TACC_FLOATING_IP $TACC_FLOATING_IP 


TACC DirectStitch VLAN: 3505
TACC_FLOATING_IP 129.114.109.236


#### Add a subnet and router to the network

In [13]:
OS_REGION_NAME='CHI@TACC'
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --allocation-pool start=${TACC_ALLOCATION_START},end=${TACC_ALLOCATION_END} \
                        --dhcp \
                        --network ${TACC_NETWORK_NAME} \
                        ${TACC_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${TACC_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${TACC_ROUTER_NAME} ${TACC_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${TACC_ROUTER_NAME}

echo Network ${TACC_NETWORK_NAME} is ready for nodes!

Creating Subnet
+-------------------+----------------------------------------------------------+
| Field             | Value                                                    |
+-------------------+----------------------------------------------------------+
| allocation_pools  | 192.168.100.20-192.168.100.29                            |
| cidr              | 192.168.100.0/24                                         |
| created_at        | 2019-05-30T02:03:36Z                                     |
| description       |                                                          |
| dns_nameservers   |                                                          |
| enable_dhcp       | True                                                     |
| gateway_ip        | 192.168.100.1                                            |
| host_routes       |                                                          |
| id                | a250768f-7442-4e0c-a334-a7fb4408c49d                     |
| ip_version

#### Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [14]:
echo TACC_RESERVATION $TACC_RESERVATION

TACC_NODE_NAME=${RESOURCE_PREFIX}-node

OS_REGION_NAME='CHI@TACC'
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-CentOS7" \
                        --key-name ${SSH_KEY_NAME} \
                        --hint reservation=${TACC_NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${TACC_NETWORK_NAME} \
                        ${TACC_NODE_NAME}


TACC_RESERVATION 72fdd6ab-334e-458b-b2d5-3f16c1e768ab
Creating servers... This will take several minutes!
+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         |                                        |
| OS-EXT-SRV-ATTR:host                | None                                   |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                   |
| OS-EXT-SRV-ATTR:instance_name       |                                        |
| OS-EXT-STS:power_state              | NOSTATE                                |
| OS-EXT-STS:task_state               | scheduling                             |
| OS-EXT-STS:vm_state                 | building                               |
| O

In [20]:
OS_REGION_NAME='CHI@TACC'

#Attach the floating IP address to the server
echo $TACC_FLOATING_IP 
openstack server add floating ip $TACC_NODE_NAME $TACC_FLOATING_IP 

129.114.109.236


## Step 2: Use ExoGENI to Connect the Ports


In [19]:
echo UC DirectStitch VLAN:   $UC_DIRECTSTITCH_VLAN
echo TACC DirectStitch VLAN: $TACC_DIRECTSTITCH_VLAN
xoStitch create -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE

UC DirectStitch VLAN: 3293
TACC DirectStitch VLAN: 3505
Opening certificate ~/work/geni-pruth.pem and key ~/work/geni-pruth.pem
Exception in thread "main" org.renci.ahab.libtransport.util.UtilTransportException: SSHAccessTokenFileFactory unable to load user public ssh key /home/pruth/.ssh/id_rsa.pub
	at org.renci.ahab.libtransport.util.SSHAccessTokenFileFactory.<init>(SSHAccessTokenFileFactory.java:39)
	at org.renci.xostitch.XoStitch.buildContext(XoStitch.java:307)
	at org.renci.xostitch.XoStitch.main(XoStitch.java:271)


: 1

## Step 3: Run Experiments


## Experiment with the DirectStitch Connection

Add a floating IP to the nodes and ssh to them in separate windows.

- Ping from one node to the other using the fixed IPs. What is the RTT?
- Use traceroute find the path between the nodes. How many hops do you see if you use the fixed IPs? How many hops uing the floating IP? 
- Use iperf3 to test the bandwidth between the nodes. Set the MTU to use jumbo frames. Does the bandwidth change?

## Other Useful Commands

In [ ]:
#Get the status of the ExoGENI circuit
xoStitch status -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE


In [ ]:
#Delete the ExoGENI circuit
xoStitch delete -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE


### Clean Up!

In [ ]:
OS_REGION_NAME='CHI@UC'
openstack server delete --wait ${UC_NODE_NAME}
blazar lease-delete $UC_RESERVATION

In [ ]:
OS_REGION_NAME='CHI@TACC'
openstack server delete --wait ${TACC_NODE_NAME}
blazar lease-delete $TACC_RESERVATION

In [ ]:
#Delete the ExoGENI circuit
xoStitch delete -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE
